In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
df = pd.read_csv('etl_df', low_memory= False)
df.head()

,original_language,overview,popularity,release_date,status,tagline,title,vote_average,anio,return_(%),runtime_(minutos),budget_(dolares),revenue_(dolares),director,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,Released,NaN,Toy Story,7.7,1995.0,1245.0,81.0,30000000.0,373554033.0,John Lasseter,Toy Story Collection,Animation - Comedy - Family,Pixar Animation Studios,United Statesof America,English
1,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,1995.0,404.0,104.0,65000000.0,262797249.0,Joe Johnston,NaN,Adventure - Fantasy - Family,Tri Star Pictures - Teitler Film - Interscop...,United Statesof America,English - Français
2,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,1995.0,0.0,101.0,0.0,0.0,Howard Deutch,Grumpy Old Men Collection,Romance - Comedy,Warner Bros. - Lancaster Gate,United Statesof America,English
3,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,1995.0,509.0,127.0,16000000.0,81452156.0,Forest Whitaker,NaN,Comedy - Drama - Romance,Twentieth Century Fox Film Corporation,United Statesof America,English
4,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,1995.0,0.0,106.0,0.0,76578911.0,Charles Shyer,Fatherofthe Bride Collection,Comedy,Sandollar Productions - Touchstone Pictures,United Statesof America,English


In [52]:
def peliculas_idioma(idioma: str):
    cont = df['original_language'].value_counts()[idioma]
    
    return print(cont, 'peliculas se estrenaron en', idioma)

In [53]:
peliculas_idioma('es')

992 peliculas se estrenaron en es


In [54]:
def peliculas_duracion(titulo: str):
    dato = titulo.title()
    pelicula = df.loc[df.index[df['title'] == dato].tolist(), 'title'].iloc[0]
    duracion = df.loc[df.index[df['title'] == dato].tolist(), 'runtime_(minutos)'].iloc[0]
    anio = df.loc[df.index[df['title'] == dato].tolist(), 'anio'].iloc[0]
    
    return pelicula, duracion, anio

In [55]:
peliculas_duracion('JUMANJI')

('Jumanji', 104.0, 1995.0)

In [56]:
def franquicia(franquicia:str):
    dato = franquicia.title()
    df["belongs_to_collection"] = df["belongs_to_collection"].fillna('')
    franq = df[df["belongs_to_collection"].str.contains(dato, case=False)]
    cantidad = franq['belongs_to_collection'].count()
    ganancia_total = round(franq['revenue_(dolares)'].sum() - franq['budget_(dolares)'].sum())
    ganancia_promedio = round(franq['revenue_(dolares)'].mean() - franq['budget_(dolares)'].mean())
    if franq.empty:
       error = {'error': "f'{dato} parametro incorrecto"}
       return error

    return dato, cantidad, ganancia_total, ganancia_promedio

    #return {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}

In [57]:
franquicia('Toy Story')

('Toy Story', 3, 1617890605, 539296868)

In [58]:
def peliculas_pais(pais:str):
    dato = pais.title()
    df["production_countries"] = df["production_countries"].fillna('')
    countr = df[df["production_countries"].str.contains(dato, case=False)]
    respuesta = countr['production_countries'].count()
    if countr.empty:
       error = {'error': "f'{dato} parametro incorrecto"}
       return error

    return dato, respuesta
    #return {'pais':pais, 'cantidad':respuesta}


In [59]:
peliculas_pais('argentina')

('Argentina', 254)

In [60]:
def productoras_exitosas(productora:str):
    dato = productora.title()
    df["production_companies"] = df["production_companies"].fillna('')
    prod = df[df["production_companies"].str.contains(dato, case=False)]
    revenue_total = round(prod['revenue_(dolares)'].sum())
    cantidad = prod['title'].count()
    if prod.empty:
       error = {'error': "f'{dato} parametro incorrecto"}
       return error

    return dato, revenue_total, cantidad
    # return {'productora':productora, 'revenue_total': respuesta,'cantidad':respuesta}

In [61]:
productoras_exitosas('Warner Bros')

('Warner Bros', 67083668593, 1372)

In [62]:
def get_director(nombre_director:str):
    dato = nombre_director.title()
    dir = df[df['director'] == dato]
    retorno_total_director = dir['revenue_(dolares)'].sum()
    peliculas = dir['title'].unique()
    anio = dir['anio']
    retorno_pelicula = dir['return_(%)']
    budget_pelicula = dir['budget_(dolares)']
    revenue_pelicula = dir['revenue_(dolares)']
    if dir.empty:
       error = {'error': "f'{dato} parametro incorrecto"}
       return error

    return [dato, retorno_total_director, peliculas, anio, retorno_pelicula, budget_pelicula, revenue_pelicula]
    ''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma. En formato lista'''
    
    # return {'director':nombre_director, 'retorno_total_director':respuesta, 
    #'peliculas':respuesta, 'anio':respuesta,, 'retorno_pelicula':respuesta, 
    #'budget_pelicula':respuesta, 'revenue_pelicula':respuesta}

In [63]:
get_director('John Lasseter')

['John Lasseter',
 2256015306.0,
 array(['Toy Story', "A Bug's Life", 'Toy Story 2', 'Luxo Jr.', 'Cars',
        'Cars 2', 'Tin Toy', "Red's Dream", 'Knick Knack',
        'Mater and the Ghostlight'], dtype=object),
 0        1995.0
 2241     1998.0
 2997     1999.0
 10662    1986.0
 10982    2006.0
 17356    2011.0
 19102    1988.0
 19156    1987.0
 19206    1989.0
 22825    2006.0
 Name: anio, dtype: float64,
 0        1245.0
 2241      303.0
 2997      553.0
 10662       0.0
 10982     385.0
 17356     280.0
 19102       0.0
 19156       0.0
 19206       0.0
 22825       0.0
 Name: return_(%), dtype: float64,
 0         30000000.0
 2241     120000000.0
 2997      90000000.0
 10662            0.0
 10982    120000000.0
 17356    200000000.0
 19102            0.0
 19156            0.0
 19206            0.0
 22825            0.0
 Name: budget_(dolares), dtype: float64,
 0        373554033.0
 2241     363258859.0
 2997     497366869.0
 10662            0.0
 10982    461983149.0
 17356   

In [64]:
def recomendacion(titulo:str):
    dato = titulo.title()
    #Extraer el titulo y el genero
    primer_coincidencia_df = df[df["title"].str.contains(dato, case=False)]
    if primer_coincidencia_df.empty:
        error = {'error': "f'{dato} parametro incorrecto"}
        return error 

    titulo = primer_coincidencia_df['title'].iloc[0]
    genero = primer_coincidencia_df['genres'].iloc[0]
    sep = genero.find(' ')
    genero = genero[:sep]

    #Reducir el df a los que contengan el genero
    df['genres'] = df['genres'].fillna('')
    df_reducido = df[df["genres"].str.contains(genero, case=False)]
    
    #Eliminar columnas que no utilizaremos
    col_eliminar = ['original_language', 'release_date', 'status', 'tagline', 'runtime_(minutos)',
                    'budget_(dolares)', 'revenue_(dolares)', 'production_companies', 'production_countries']
    df_reducido.drop(columns= col_eliminar, inplace=True)

    #Splitear overview
    sp_over = df_reducido['overview'].str.split(pat= ' ', n= -1, expand= True)
    df_reducido = pd.concat([df_reducido, sp_over], axis= 1)
    df_reducido.drop(columns='overview', inplace= True)

    #Splitear spoken_languages
    sp_lang = df_reducido['spoken_languages'].str.split(pat= '-', n= -1, expand= True)
    df_reducido = pd.concat([df_reducido, sp_lang], axis= 1)
    df_reducido.drop(columns='spoken_languages', inplace= True)

    #Separar la columna title, obtener el indice del dato de entrada y eliminar la columna title del df reducido al cual se aplicara encoding
    lista_title = df_reducido['title'] #El indice de la variable X es [0]
    df_reducido.drop(columns='title', inplace= True)
    
    #Convertir todo df_reducido a tipo de dato string
    df_reducido = df_reducido.astype(str)

    #Aplicar label encoding a df_reducido
    label_encoder = LabelEncoder()
    df_encoded = df_reducido.apply(label_encoder.fit_transform)

    #Similitud de coseno a df_encoded
    similitud = cosine_similarity(df_encoded)

    #Debido a que el indice del dato de entrada es 0, separamos la primera fila de la matriz en un array

    array_similitud = similitud[0]

    #Obtener los indices de los 6 valores mas altos
    indices = np.argsort(-array_similitud)[:6]

    #Creamos una lista vacia 'respuesta', iteramos los indices y agregamos el valor de lista_title para cada indice, quitandole el indice [0], esas seran las peliculas recomendadas.
    respuesta = []

    for elemento in indices:
        respuesta.append(lista_title.iloc[elemento])

    respuesta = respuesta[1:]
   
    return titulo, respuesta

In [65]:
recomendacion('toy story')


C:\Users\eduen\AppData\Local\Temp\ipykernel_14532\2675785078.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reducido.drop(columns= col_eliminar, inplace=True)


('Toy Story',
 ['Mind Game',
  'A Close Shave',
  'Animals United',
  'Ice Age: The Meltdown',
  'Megamind: The Button Of Doom'])